# Tree-of-Thought

- 원래의 ToT 논문은 Game of 24, Mini Crossword, 4개의 문장을 인위적으로 뽑아 글을 작성하도록 하는 태스크
- 현실 어플리케이션이 아니라 AI의 지적능력을 확인하고 싶었던 것이기에 실제 사용하려면 응용이 필요함ㅇ

In [1]:
from typing import Union
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser, StrOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
import pandas as pd

/Users/hyeonjinho/.pyenv/versions/3.11.6/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3577: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


In [2]:
eval_model = ChatOpenAI(model="gpt-4-1106-preview", temperature=0.8) # CoT 는 다양한 샘플을 만들어야하기 때문에 temperature를 올려야함
basic_model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.8) # CoT 는 다양한 샘플을 만들어야하기 때문에 temperature를 올려야함

## PlanGenerationChain

In [3]:
plan_gen_prompt = ChatPromptTemplate.from_messages(
    [
        ('human', "{subject}에 대한 에세이를 쓰고싶어. 이 주제에 대해 에세이를 쓰기 위해 계획을 세워줘"),
    ])

In [4]:
plan_gen_chain = plan_gen_prompt | basic_model | StrOutputParser()

In [5]:
plan = plan_gen_chain.invoke({"subject": "AI로 인한 데이팅 업계의 변화"})
print(plan)

제목: AI로 인한 데이팅 업계의 혁신과 도전

1. 소개
- 데이팅 업계는 인간 간의 관계 형성을 돕는 중요한 산업으로 발전해왔다. 그러나 최근 AI 기술의 발전으로 데이팅 업계는 새로운 변화를 맞이하고 있다.
- 본 에세이에서는 AI로 인한 데이팅 업계의 혁신과 도전에 대해 탐구하고자 한다.

2. AI의 역할과 혁신
- AI 기술은 데이팅 앱과 웹사이트에서 많은 혁신을 이끌어내고 있다. 예를 들어, AI 기반의 매칭 알고리즘을 통해 사용자들에게 더욱 정확한 매칭을 제공하고 있다.
- 또한, AI는 사용자들의 행동과 취향을 분석하여 개인 맞춤형 서비스를 제공하고, 대화 시뮬레이션을 통해 데이트 전략을 개선하는 등 다양한 분야에서 활약하고 있다.

3. 도전과 해결책
- 한편, AI로 인한 데이팅 업계의 발전은 도전과 함께 따르고 있다. 예를 들어, 개인정보 보호 문제와 AI가 인간의 감정을 이해하는 한계 등이 있다.
- 이러한 도전에 대처하기 위해서는 AI 개발자와 업체들이 더욱 더 윤리적인 AI를 개발하고, 사용자들의 프라이버시를 존중하는 방향으로 나아가야 한다.

4. 결론
- AI로 인한 데이팅 업계의 혁신은 데이팅 경험을 더욱 풍요롭고 효율적으로 만들어주고 있다. 그러나 동시에 도전도 함께 따르고 있어 해결해야 할 과제들이 많이 남아있다.
- 따라서, AI 기술의 발전과 데이팅 업계의 혁신을 위해 우리는 윤리적이고 안전한 AI를 발전시키는 노력을 계속해야 한다.

5. 참고문헌
- 관련 논문, 보고서, 기사 등을 인용하여 보다 신뢰성 있는 에세이를 완성한다.


## Voting Chain

In [6]:
class VoteCoT(BaseModel):
    thought: str = Field(description="voting number를 선택한 이유에 대해 자세히 넣어주세요.")
    voting_num: int = Field(description="voting number")

In [7]:
parser = JsonOutputParser(pydantic_object=VoteCoT)
format_instructions = parser.get_format_instructions()
format_instructions

'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"thought": {"title": "Thought", "description": "voting number를 선택한 이유에 대해 자세히 넣어주세요.", "type": "string"}, "voting_num": {"title": "Voting Num", "description": "voting number", "type": "integer"}}, "required": ["thought", "voting_num"]}\n```'

In [8]:
voting_prompt_template = HumanMessagePromptTemplate.from_template(
                            "아래는 에세이 작성을 위한 계획 또는 에세이 글이다. 아래의 후보 중 가장 좋은 것을 추론 과정과 함께 투표 번호를 응답해.\n {candidates}\n{format_instructions}",
                            partial_variables={"format_instructions": format_instructions})

voting_prompt = ChatPromptTemplate.from_messages(
    [
        voting_prompt_template,
    ])

In [9]:
voting_chain = voting_prompt | eval_model | parser

In [10]:
plan_list = plan_gen_chain.batch([{"subject": "AI로 인한 데이팅 업계의 변화"}] * 2)


In [11]:
print(plan_list[0])

에세이를 쓰기 위해 다음과 같은 계획을 세울 수 있을 것입니다.

1. 서론
- 데이팅 업계의 중요성과 인기 이유 소개
- AI 기술의 발전으로 인한 데이팅 업계의 변화 소개

2. 데이팅 업계의 전통적인 모습
- 데이팅 앱과 웹사이트의 등장과 성공적인 사례 소개
- 온라인 데이팅의 장단점과 사용자들의 반응 분석

3. AI가 데이팅 업계에 미친 영향
- AI 기술을 활용한 매칭 알고리즘의 등장과 효과 소개
- AI를 통해 개인 맞춤형 데이팅 서비스 제공 가능성 분석

4. AI로 인한 데이팅 업계의 변화와 이슈
- AI를 통한 데이팅 시스템의 효율성과 정확도 분석
- AI가 데이팅 업계에 미치는 사회적, 윤리적 문제 등에 대한 논의

5. 결론
- AI가 데이팅 업계에 미친 변화와 영향 정리
- 데이팅 업계의 미래 전망 및 AI 기술의 활용 가능성에 대한 전망과 제언

이러한 계획을 바탕으로 흥미로운 주제와 깊은 분석을 통해 AI가 데이팅 업계에 미친 변화에 대해 포괄적으로 다룬 에세이를 작성할 수 있을 것입니다.


In [12]:
print(plan_list[1])

1. 소개: AI 기술이 빠르게 발전하면서 데이팅 업계에도 큰 변화가 일어나고 있다. 이전에는 사람들이 직접 만남을 찾아야 했지만, 이제는 AI가 매칭을 도와주고 상대방을 추천해주는 시대가 되었다. 이에 대한 장단점을 살펴보고자 한다.

2. AI의 역할: AI는 사용자의 프로필을 분석하여 적합한 상대방을 추천해주는 역할을 한다. 또한, 사용자의 취향이나 성향을 파악하여 맞춤형 데이트 아이디어를 제공하기도 한다. 이를 통해 보다 효율적인 데이팅이 가능해지고, 시간과 에너지를 절약할 수 있다.

3. 장점: AI를 통한 데이팅은 더 많은 사람들과 소통할 수 있는 기회를 제공한다. 또한, AI는 객관적인 정보를 제공하기 때문에 주관적인 판단을 배제할 수 있어 효율적인 결정을 내릴 수 있다. 또한, AI는 사용자의 성향을 파악하여 적합한 상대방을 추천하기 때문에 보다 더 깊은 관계를 형성할 수 있다.

4. 단점: 하지만, AI를 통한 데이팅은 사람 간의 진짜 소통을 대신할 수는 없다. 인간적인 감정과 상호작용은 AI로 대체할 수 없기 때문에 완전히 의존하면 안된다. 또한, AI가 사용자의 정보를 수집하고 분석하기 때문에 개인정보 보호 문제도 고려해야 한다.

5. 결론: AI가 데이팅 업계에 미치는 영향은 두 가지 면에서 볼 수 있다. 한편으로는 효율적인 매칭과 맞춤형 서비스를 제공하여 사용자들에게 편의성을 제공한다. 다른 한편으로는 인간적인 소통과 감정을 대체할 수 없기 때문에 조심스럽게 활용해야 한다. 최종적으로, AI를 통한 데이팅은 보다 다양하고 효율적인 만남을 찾을 수 있는 새로운 방법이 될 수 있을 것이다.


In [13]:
candidates = "\n\n".join([f"후보 {i}.\n{plan}" for i, plan in enumerate(plan_list)])

In [14]:
print(candidates)

후보 0.
에세이를 쓰기 위해 다음과 같은 계획을 세울 수 있을 것입니다.

1. 서론
- 데이팅 업계의 중요성과 인기 이유 소개
- AI 기술의 발전으로 인한 데이팅 업계의 변화 소개

2. 데이팅 업계의 전통적인 모습
- 데이팅 앱과 웹사이트의 등장과 성공적인 사례 소개
- 온라인 데이팅의 장단점과 사용자들의 반응 분석

3. AI가 데이팅 업계에 미친 영향
- AI 기술을 활용한 매칭 알고리즘의 등장과 효과 소개
- AI를 통해 개인 맞춤형 데이팅 서비스 제공 가능성 분석

4. AI로 인한 데이팅 업계의 변화와 이슈
- AI를 통한 데이팅 시스템의 효율성과 정확도 분석
- AI가 데이팅 업계에 미치는 사회적, 윤리적 문제 등에 대한 논의

5. 결론
- AI가 데이팅 업계에 미친 변화와 영향 정리
- 데이팅 업계의 미래 전망 및 AI 기술의 활용 가능성에 대한 전망과 제언

이러한 계획을 바탕으로 흥미로운 주제와 깊은 분석을 통해 AI가 데이팅 업계에 미친 변화에 대해 포괄적으로 다룬 에세이를 작성할 수 있을 것입니다.

후보 1.
1. 소개: AI 기술이 빠르게 발전하면서 데이팅 업계에도 큰 변화가 일어나고 있다. 이전에는 사람들이 직접 만남을 찾아야 했지만, 이제는 AI가 매칭을 도와주고 상대방을 추천해주는 시대가 되었다. 이에 대한 장단점을 살펴보고자 한다.

2. AI의 역할: AI는 사용자의 프로필을 분석하여 적합한 상대방을 추천해주는 역할을 한다. 또한, 사용자의 취향이나 성향을 파악하여 맞춤형 데이트 아이디어를 제공하기도 한다. 이를 통해 보다 효율적인 데이팅이 가능해지고, 시간과 에너지를 절약할 수 있다.

3. 장점: AI를 통한 데이팅은 더 많은 사람들과 소통할 수 있는 기회를 제공한다. 또한, AI는 객관적인 정보를 제공하기 때문에 주관적인 판단을 배제할 수 있어 효율적인 결정을 내릴 수 있다. 또한, AI는 사용자의 성향을 파악하여 적합한 상대방을 추천하기 때문에 보다 더 깊은 관계를 형성할 수 있다.

4. 단점: 하지만, AI를

In [15]:
vote = voting_chain.invoke({"candidates": candidates})

In [16]:
vote

{'thought': 'Candidate 0 provides a more detailed and structured plan for the essay, with a clear introduction, analysis of the traditional dating industry, impact of AI, and a comprehensive conclusion that includes societal and ethical considerations. This thorough approach seems likely to result in a more in-depth and well-rounded essay.',
 'voting_num': 0}

## Essay Generation Chain

In [17]:
essay_gen_prompt = ChatPromptTemplate.from_messages(
    [
        ('human', "{plan}에 대해 제목과 함께 1000자 내외의 에세이를 작성해줘."),
    ])
essay_gen_chain = essay_gen_prompt | basic_model | StrOutputParser()

In [18]:
essay = essay_gen_chain.invoke({"plan": plan})
print(essay)

데이팅 업계는 사람들이 새로운 인연을 만들고 소통하는 중요한 플랫폼으로 자리 잡고 있다. 그러나 최근 AI 기술의 발전으로 데이팅 업계는 새로운 혁신과 도전을 맞이하고 있다. 

AI 기술은 데이팅 앱과 웹사이트에서 많은 혁신을 가져오고 있다. AI 기반의 매칭 알고리즘을 통해 사용자들에게 더욱 정확한 매칭을 제공함으로써 데이팅 경험을 개선하고 있다. 또한, AI는 사용자들의 행동과 취향을 분석하여 개인 맞춤형 서비스를 제공하고, 대화 시뮬레이션을 통해 데이트 전략을 개선하는 등의 다양한 방법으로 데이팅 업계를 혁신하고 있다.

그러나 AI로 인한 데이팅 업계의 발전은 도전과 함께 따라오고 있다. 개인정보 보호 문제와 AI가 인간의 감정을 완벽하게 이해하는 한계 등의 문제가 있다. 이러한 도전에 대처하기 위해서는 윤리적인 AI 개발과 사용자 프라이버시를 존중하는 노력이 필요하다.

AI로 인한 데이팅 업계의 혁신은 데이팅 경험을 더욱 풍요롭고 효율적으로 만들어주고 있다. 그러나 도전도 함께 따르고 있어 해결해야 할 과제들이 많이 남아있다. 따라서, AI 기술의 발전과 데이팅 업계의 혁신을 위해 우리는 윤리적이고 안전한 AI를 발전시키는 노력을 계속해야 한다. 이를 통해 더 나은 데이팅 경험과 건강한 데이팅 문화를 만들어 나갈 수 있을 것이다.


## ToT Pipeline

In [19]:
# subject 정하기
subject = "AI로 인한 데이팅 업계의 변화"

In [20]:
# plan 5개 샘플
plan_list = plan_gen_chain.batch([{"subject": subject}]*5)

In [21]:
# vote 5개로 plan 중 가장 좋은것 고르기
candidates = "\n\n".join([f"후보 {i}.\n{plan}" for i, plan in enumerate(plan_list)])
vote_list = voting_chain.batch([{"candidates": candidates}]*5)

In [22]:
df = pd.DataFrame(vote_list)

In [23]:
df

,thought,voting_num
0,"후보 2를 선택했습니다. 이 계획은 에세이의 기본적인 흐름을 잘 파악하고 있으며, ...",2
1,후보 2 is the most comprehensive and balanced pl...,2
2,I chose 후보 2 as it provides a comprehensive pl...,2
3,후보 0의 에세이 계획은 소개부터 결론까지 AI 기술이 데이팅 업계에 미친 영향을 ...,0
4,Candidate 0 was chosen because it presents a b...,0


In [24]:
best_candi_num = df['voting_num'].mode()[0]
best_candi_num

2

In [25]:
best_plan = plan_list[best_candi_num]

In [26]:
# best plan으로 essay generation
essay_list = essay_gen_chain.batch([{"plan": best_plan}]*5)

In [27]:
# vote 5개로 essay 중 가장 좋은것 고르기
candidates = "\n\n".join([f"후보 {i}.\n{essay}" for i, essay in enumerate(essay_list)])
vote_list = voting_chain.batch([{"candidates": candidates}]*5)

In [28]:
df = pd.DataFrame(vote_list)
df

,thought,voting_num
0,"후보 0 was chosen based on its clear structure, ...",0
1,"후보들의 글을 비교했을 때, 후보 0이 가장 잘 구성된 글로 보입니다. 후보 0은 ...",0
2,I selected Voting Number 0 because it offers a...,0
3,후보들의 에세이는 모두 AI 기술의 발전이 데이팅 업계에 미치는 영향에 대해 다루고...,0
4,"후보 0는 AI 기술의 발전과 데이팅 업계의 변화를 잘 설명하고 있습니다. 과거, ...",0


In [29]:
best_candi_num = df['voting_num'].mode()[0]
best_candi_num

0

In [30]:
best_essay = essay_list[best_candi_num]

In [31]:
print(best_essay)

AI 기술의 발전과 데이팅 업계

과거와 현재, 그리고 미래

요즘은 AI 기술의 발전으로 인해 우리의 일상이 많이 변화하고 있다. 특히 데이팅 업계에서도 AI 기술이 큰 변화를 가져오고 있다. 과거에는 데이팅이 소개팅이나 소모임을 통해 이뤄졌지만, 현재는 데이팅 앱과 서비스를 통해 손쉽게 데이트 상대를 만날 수 있다. 이러한 변화는 AI 기술의 발전과 함께 더욱 가속화되고 있으며, 데이팅 경험을 혁신적으로 변화시키고 있다.

과거에는 데이팅이 상대를 직접 만나서 대화를 나누는 것이 중요했지만, AI 기술이 등장함으로써 데이팅 앱은 사용자의 프로필과 행동 패턴을 분석하여 적합한 상대를 추천해주는 기능을 제공한다. 이를 통해 사용자들은 더욱 쉽고 빠르게 자신과 잘 맞는 상대를 찾을 수 있게 되었다. 또한 AI 기술은 대화를 자연스럽게 이끌어주는 챗봇을 통해 데이팅 경험을 더욱 풍부하고 흥미롭게 만들어주고 있다.

하지만 AI를 통한 데이팅 경험에는 몇 가지 단점도 존재한다. 예를 들어, 사용자의 개인정보가 AI 알고리즘에 의해 분석되고 활용되는 것에 대한 우려가 있으며, 또한 자기 표현이나 인간적인 요소가 부족해지는 경우도 있다. 이러한 부분은 사용자들에게 불편함을 느끼게 할 수 있지만, 이러한 단점을 극복하고 긍정적인 면을 최대화하기 위해 계속해서 기술과 서비스의 발전이 이루어져야 한다.

앞으로 AI 기술의 발전이 데이팅 업계에 미칠 영향은 더욱 커질 것으로 예상된다. 머신 러닝과 딥 러닝 기술의 발전으로 인해 더욱 정확하고 개인화된 매칭 시스템이 구축될 것이며, 가상 현실과 홀로그램 기술을 통해 사용자들은 더욱 현실적이고 다채로운 데이팅 경험을 할 수 있을 것이다.

결론적으로, AI 기술의 발전으로 데이팅 업계는 큰 변화를 겪고 있으며, 앞으로 더욱 혁신적인 변화가 예상된다. 사용자들은 AI를 통해 보다 쉽고 효율적으로 데이트 상대를 찾을 수 있게 되었지만, 동시에 AI에 대한 우려도 존재한다. 이러한 문제를 극복하고 긍정적인 측면을 최대화하며, 사용자들